<a href="https://colab.research.google.com/github/darinkist/MediumArticle_InteractiveChatGPTSessionsInJupyterNotebook/blob/main/ColabDemo_Medium_Article_Evaluate_Monitor_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install trulens-eval langchain -q

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR KEY HERE"
os.environ["HUGGINGFACE_API_KEY"] = "YOUR KEY HERE"
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "YOUR KEY HERE"

In [ ]:
# imports from LangChain to build app
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (ChatPromptTemplate,
                                    HumanMessagePromptTemplate)
from langchain import HuggingFaceHub

# create LLM chain
full_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template="You are a tourist guide and gourmet to provide" \
        "helpful information about the following question: {prompt}"\
        "Name at least 2 restaurants and the dishes they are famous for.",
            input_variables=["prompt"],
        )
    )
chat_prompt_template = ChatPromptTemplate.from_messages([full_prompt])

# You can choose between gpt-3.5-turbo and google/flan-t5-xxl
google = HuggingFaceHub(repo_id="google/flan-t5-xxl",
                     model_kwargs={"temperature":0.9})

chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.9)

# Provide here as a parameter value for llm the model you'd like to use
chain = LLMChain(llm=google, prompt=chat_prompt_template)

In [ ]:
from trulens_eval import Feedback, Huggingface, Query

# Initialize HuggingFace-based feedback function collection class:
hugs = Huggingface()

# Define a language match feedback function using HuggingFace.
f_lang_match = Feedback(hugs.language_match).on(
    text1=Query.RecordInput, text2=Query.RecordOutput
)

# Check if model's answer is toxic
f_toxity = Feedback(hugs.not_toxic).on(text=Query.RecordOutput)

In [ ]:
from trulens_eval import TruChain

truchain = TruChain(
    chain,
    app_id='TestApp-ABC',
    feedbacks=[f_lang_match, f_toxity]
)

In [ ]:
truchain("Where can I find the best tapas in Barcelona?")

# COLAB WORKAROUND
* If you want to launch the dashboard from Colab, please run the cell below.
* More information can be found [here](https://alphasec.io/run-a-streamlit-app-with-google-colab-notebook/)

In [ ]:
# Colab workaround
!npm install localtunnel

In [ ]:
from trulens_eval import Tru
tru = Tru()
tru.start_dashboard()

In [ ]:
# Colab workaround
# Click on the generated link and follow the instructions
# Enter your "External URL" (only the IP without the http:// or port) in the respective form
!npx localtunnel --port 8501

In [ ]:
tru.stop_dashboard()